# Segmenting Cells in Microscopy Image

This notebook aims to segment cells in a provided microscopy image and measure their areas using the watershed algorithm.

## Step 1: Load and Preprocess the Image
First, we load the image and convert it to grayscale if necessary.

In [ ]:
from skimage import io, color
import matplotlib.pyplot as plt

# Load the image
image = io.imread('../data/membranes.png')

# Convert to grayscale if necessary
if len(image.shape) == 3:
    image_gray = color.rgb2gray(image)
else:
    image_gray = image

# Display the original image
plt.imshow(image_gray, cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')
plt.show()

## Step 2: Apply the Watershed Algorithm
Next, we compute the local gradient of the image and apply the watershed algorithm.

In [ ]:
from skimage import filters, morphology, segmentation
from scipy import ndimage as ndi

# Compute the local gradients
gradient = filters.sobel(image_gray)

# Generate markers
markers = ndi.label(image_gray < filters.threshold_otsu(image_gray))[0]

# Apply watershed
labeled_image = segmentation.watershed(gradient, markers)

# Display the watershed result
plt.imshow(labeled_image, cmap='nipy_spectral')
plt.title('Watershed Segmented Image')
plt.axis('off')
plt.show()

## Step 3: Clean and Analyze the Segmented Regions
We remove small objects from the labeled image and measure the remaining objects' areas.

In [ ]:
from skimage import measure
import pandas as pd

# Remove small objects
cleaned_image = morphology.remove_small_objects(labeled_image, min_size=150)

# Measure properties of labeled regions
properties = measure.regionprops_table(cleaned_image, properties=['label', 'area'])

df = pd.DataFrame(properties)
df.head()

## Step 4: Visualize and Interpret Results
Finally, we visualize the cleaned regions and print out the areas of each segmented cell.

In [ ]:
from skimage import color

# Display the cleaned image with color labels
plt.imshow(color.label2rgb(cleaned_image, image=image_gray, bg_label=0), cmap='nipy_spectral')
plt.title('Cleaned Segmented Cells')
plt.axis('off')
plt.show()

# Print the areas of the segmented cells
print(df)